# Liquidity Measure


## Summary
We need to measure liquidity available on a market in order to see whether market makers are keeping 
their commitment. 
Here we propose a method that counts liquidity as the probability weighted average of the value of volume on the book (in the settlement currency of the market). 
This gives view of liquidity at one instant of time; we then use exponential weighted average over time to obtain the desired measure.

## Inputs
- network parameter `market.liquidity.probabilityOfTrading.tau.scaling` which has to be $>10^{-4}$.
- risk model for the market but with $\tau$ (`tau`) replaced with `market.liquidity.probabilityOfTrading.tau.scaling x tau`.
- order book volume

## Terminology
- Instantenaous liquidity $\lambda$, defined below in detail in this spec file.
- `mid price = (best bid - best offer) / 2` (or undefined if either side of the book is empty).
- buy / sell side volume refer to the volume available at a distance from mid price, $V = V(x)$, where $x > 0$ refers to sell side, $x < 0$ refers to buy side and $x$ belongs to the set of all price points available on the book.
- probability of volume at distance from mid price trading: $p = p(x)$, this comes from risk model with $\tau$ (`tau`) replaced with `market.liquidity.probabilityOfTrading.tau.scaling x tau`.
- auction level buy price $x_{min} < 0$ and auction level sell price $x_{max} > 0$ for will come from auction spec.

Note that both $\delta$ and $\alpha$  are network wide parameters.


## Details
During auctions (price monitoring auctions, liquidity auctions, frequent batch auctions) we don't have `mid price` as mentioned above. Use the `last trade price` or `indicative uncrossing price` instead. 

The instantenaous liquidiy should get calculated as needed and will change with any changes to order book.

### Understanding probability of trading 
Each risk model gives probability distribution of future price moves, given current price (the risk models are too crude to care whether this is mark,  mid-price, or best bid / best ask. Given current price level $S$ and a time step $\tau > 0$ (expressed as year fraction) the risk model gives, for each "tick" the probability that the price at time $\tau$ from now will be $S_\tau$. Write this is $\mathbb P(S_\tau = x | S) =: f(x)$. The probability that volume at tick $x$ trades is 
$$
p(x) = \frac1Z\sum_{x \leq x' \leq x_{max}} f(x') \quad \text{for $x>S$ and } \quad p(x) = \frac1Z\sum_{x_{min} \leq x' \leq x} f(x') \quad \text{for $x<S$}\,,
$$
where the normalising factor $Z$ is  
$$
Z = \sum_{S \leq x' \leq x_{max}} f(x') \quad \text{for $x>S$ and } \quad Z = \sum_{x_{min} \leq x' \leq S} f(x')   \quad \text{for $x<S$}\,.
$$
We adopt the convention that $p(x) = \frac12$ for x between (and including) best bid and best ask and we normalise p(x) so that 0 \leq p(x) \leq \frac12 \forall x. 

### Calculating the instantenaous liquidity

Case 1: no mid price (i.e. best bid or best offer or both are missing)
$\lambda := 0$ if there is no mid price (i.e. when either the buy or sell side of the book are empty)

Case 2: we have best bid and best ask (or indicative uncrossing price for auctions)
- Obtain $x_{min}$ and $x_{max}$ from the price monitoring information for the nearest-in-time trigger. 
- Get the list of possible ticks $x$ s.t. $x_{max} \geq x \geq x_{best\, ask}$ values from the order book. Call these $x^+_i$, with $i = 1,\ldots,N^+$. 
- Get the list of possible ticks $x$ s.t. $x_{min} \leq x \leq x_{best\, bid}$ from the order book and call these $x^-_i$, with $i = 1, \ldots , N^-$. 
- Get the volume $V(x)$ available at each $x = x^-_i$ and $x^+_i$ from the order book.
- Get the price $S(x)$ corresponding to each $x = x^-_i$ and $x^+_i$. For futures this is immediate from the order book. For products that don't trade on price convert this to a monetary value first (e.g. if options are quoted using BS vol then convert this to option price or if a product that trades on yield, convert to price).
- Get the probability $p(x)$ of trading for each of $x = x^-_i$ and $x^+_i$ from the risk model as described above. 


Now you can calculate 
$$
\lambda := 
\min\left(
    \sum_{i=1}^{N^+} S(x^+_i) V(x^+_i) p(x^+_i), 
    \sum_{i=1}^{N^-} S(x^-_i) V(x^-_i) p(x^-_i) 
\right)\,.
$$

### Calculating the liquidity provided by a single party

This is the same as above except that $V = V(x)$ isn't the entire volume on the book at that time but rather it's the volume provided by the given participant. The best bid / best ask used are the ones for the entire order book, not the one implied by a single party.



## Acceptance Criteria

Volume implied by the liquidity provision order is that given by [0034-PROB-liquidity_measure.feature](https://github.com/vegaprotocol/vega/blob/develop/integration/features/verified/0034-PROB-liquidity_measure.feature) The feature test has covered following scenarios:

1. Orders are correctly cumulated in order book's total size(<a name="0034-PROB-002" href="#0034-PROB-002">0034-PROB-002</a>). For product spot: (<a name="0034-PROB-003" href="#0034-PROB-003">0034-PROB-003</a>);

2. Probability of trading decreases away from the mid-price (<a name="0034-PROB-005" href="#0034-PROB-005">0034-PROB-005</a>). For product spot: (<a name="0034-PROB-008" href="#0034-PROB-008">0034-PROB-008</a>).

3. Change of `market.liquidity.probabilityOfTrading.tau.scaling` will immediately change the scaling parameter, hence will change the probability of trading used for comparing quality of committed liquidity.  (<a name="0034-PROB-004" href="#0034-PROB-004">0034-PROB-004</a>). For product spot: (<a name="0034-PROB-010" href="#0034-PROB-010">0034-PROB-010</a>).



